In [1]:
# directly install the bttai files, will be used later for kaggle submission 
# REFERENCED THIS CODE FOR MODEL https://github.com/deepmancer/resnet-cifar-classification/blob/master/ResNet-18/CIFAR10_Classification.ipynb
!pip install kaggle
!kaggle competitions download bttai-ajl-2025

Traceback (most recent call last):
  File "/usr/local/bin/kaggle", line 5, in <module>
    from kaggle.cli import main
  File "/usr/local/lib/python3.10/dist-packages/kaggle/__init__.py", line 7, in <module>
    api.authenticate()
  File "/usr/local/lib/python3.10/dist-packages/kaggle/api/kaggle_api_extended.py", line 407, in authenticate
    raise IOError('Could not find {}. Make sure it\'s located in'
OSError: Could not find kaggle.json. Make sure it's located in /root/.config/kaggle. Or use the environment method. See setup instructions at https://github.com/Kaggle/kaggle-api/


## Note - About file path

You could use the cell above to print the names of the file directories and get the following directories:

```
/kaggle/input/bttai-ajl-2025/sample_submission.csv
/kaggle/input/bttai-ajl-2025/train.csv
/kaggle/input/bttai-ajl-2025/test.csv
/kaggle/input/bttai-ajl-2025/test/test/e0374ae6c1362ff183cfba28ded5421b.jpg
/kaggle/input/bttai-ajl-2025/test/test/437159c605260bdd079af230566af291.jpg
...
...
/kaggle/input/bttai-ajl-2025/train/train/dermatomyositis/11271bdf2598afdd4260db3125e1f6a5.jpg
/kaggle/input/bttai-ajl-2025/train/train/dermatomyositis/732819951dcf2b53d15ea7b8bb123b71.jpg
/kaggle/input/bttai-ajl-2025/train/train/dermatomyositis/6dcc7a8abb5e1c6e670101f4b6231246.jpg
/kaggle/input/bttai-ajl-2025/train/train/dermatomyositis/e63c3b3f0ab8905e204fe467cc7411f9.jpg
...
...
```



## 1. Import Necessary Libraries

In [2]:
# 1. Import Necessary Libraries
import pandas as pd
import numpy as np
import os
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Dropout
from tensorflow.keras.callbacks import EarlyStopping
import keras
from keras import Sequential
from keras import layers
from keras.layers import Conv2D,MaxPool2D,Dense,Flatten,BatchNormalization,Dropout, Input
from keras.optimizers import Adam
from PIL import Image
from tensorflow import data as tf_data
import matplotlib.pyplot as plt
from keras.preprocessing import image

from sklearn.model_selection import train_test_split
import torch
import torch.nn as nn
import torch.nn.functional as F
from torchvision import transforms,models,datasets
import matplotlib.pyplot as plt
from PIL import Image
import numpy as np

from torch import optim
device = 'cuda' if torch.cuda.is_available() else 'cpu'
from torch.utils.data import DataLoader, Dataset
import torch.optim as optim
import torch.optim.lr_scheduler as lr_scheduler
from torch.optim.lr_scheduler import _LRScheduler
import torch.utils.data as data
import random
import warnings
import torchvision.transforms as transforms
import torchvision.datasets as datasets
import torchvision.models as models

from sklearn import decomposition
from sklearn import manifold
from sklearn.metrics import confusion_matrix
from sklearn.metrics import ConfusionMatrixDisplay

import copy
from collections import namedtuple
import os
import random
import shutil
import time

from tqdm import tqdm
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Dropout
from tensorflow.keras.callbacks import EarlyStopping
model = torch.hub.load('pytorch/vision:v0.10.0', 'resnet18', pretrained=True)


Downloading: "https://github.com/pytorch/vision/zipball/v0.10.0" to /root/.cache/torch/hub/v0.10.0.zip
/usr/local/lib/python3.10/dist-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=ResNet18_Weights.IMAGENET1K_V1`. You can also use `weights=ResNet18_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)
Downloading: "https://download.pytorch.org/models/resnet18-f37072fd.pth" to /root/.cache/torch/hub/checkpoints/resnet18-f37072fd.pth
100%|██████████| 44.7M/44.7M [00:00<00:00, 156MB/s] 


In [3]:
print(device)
warnings.filterwarnings('ignore')

cuda


## 2. Load Data

Make sure to verify the file paths if you're running on a different platform.

In [4]:
# 2. Load Data
train_df = pd.read_csv('/kaggle/input/bttai-ajl-2025/train.csv')
test_df = pd.read_csv('/kaggle/input/bttai-ajl-2025/test.csv')

# Add .jpg extension to md5hash column to reference the file_name
train_df['md5hash'] = train_df['md5hash'].astype(str) + '.jpg'
test_df['md5hash'] = test_df['md5hash'].astype(str) + '.jpg'

# Combine label and md5hash to form the correct path
train_df['file_path'] = train_df['label'] + '/' + train_df['md5hash']


# ALSO TRAIN AND TEST DIRECTORIES 
# need to change to train_dir = '/kaggle/input/bttai-ajl-2025/train/train/' for kaggle
train_dir = '/kaggle/input/bttai-ajl-2025/train/train'
test_dir = '/kaggle/input/bttai-ajl-2025/test/test'

In [5]:
# Check the first few rows to understand the structure
train_df.head()

,md5hash,fitzpatrick_scale,fitzpatrick_centaur,label,nine_partition_label,three_partition_label,qc,ddi_scale,file_path
0,fd06d13de341cc75ad679916c5d7e6a6.jpg,4,4,prurigo-nodularis,benign-epidermal,benign,NaN,34,prurigo-nodularis/fd06d13de341cc75ad679916c5d7...
1,a4bb4e5206c4e89a303f470576fc5253.jpg,1,1,basal-cell-carcinoma-morpheiform,malignant-epidermal,malignant,NaN,12,basal-cell-carcinoma-morpheiform/a4bb4e5206c4e...
2,c94ce27e389f96bda998e7c3fa5c4a2e.jpg,5,5,keloid,inflammatory,non-neoplastic,1 Diagnostic,56,keloid/c94ce27e389f96bda998e7c3fa5c4a2e.jpg
3,ebcf2b50dd943c700d4e2b586fcd4425.jpg,3,3,basal-cell-carcinoma,malignant-epidermal,malignant,NaN,34,basal-cell-carcinoma/ebcf2b50dd943c700d4e2b586...
4,c77d6c895f05fea73a8f3704307036c0.jpg,1,1,prurigo-nodularis,benign-epidermal,benign,NaN,12,prurigo-nodularis/c77d6c895f05fea73a8f37043070...


## 3. Data Preprocessing


This section demonstrates basic preprocessing techniques. To enhance data quality and model performance, consider incorporating more advanced preprocessing methods.

For further guidance, feel free to take a look at the [Image Preprocessing tutorial](https://colab.research.google.com/drive/1-ItNcRMbZBE6BCwPT-wD8m3YmHqwHxme?usp=sharing)  available in the 'Resources' section of this Kaggle competition.


In [6]:
# Drop DDI and qc for null values cols 

train_df = train_df.drop('ddi_scale', axis=1)
train_df = train_df.drop('qc', axis=1)
# Display
train_df.head(10)

,md5hash,fitzpatrick_scale,fitzpatrick_centaur,label,nine_partition_label,three_partition_label,file_path
0,fd06d13de341cc75ad679916c5d7e6a6.jpg,4,4,prurigo-nodularis,benign-epidermal,benign,prurigo-nodularis/fd06d13de341cc75ad679916c5d7...
1,a4bb4e5206c4e89a303f470576fc5253.jpg,1,1,basal-cell-carcinoma-morpheiform,malignant-epidermal,malignant,basal-cell-carcinoma-morpheiform/a4bb4e5206c4e...
2,c94ce27e389f96bda998e7c3fa5c4a2e.jpg,5,5,keloid,inflammatory,non-neoplastic,keloid/c94ce27e389f96bda998e7c3fa5c4a2e.jpg
3,ebcf2b50dd943c700d4e2b586fcd4425.jpg,3,3,basal-cell-carcinoma,malignant-epidermal,malignant,basal-cell-carcinoma/ebcf2b50dd943c700d4e2b586...
4,c77d6c895f05fea73a8f3704307036c0.jpg,1,1,prurigo-nodularis,benign-epidermal,benign,prurigo-nodularis/c77d6c895f05fea73a8f37043070...
5,9d5a90fa3f6934608add10e698001760.jpg,3,5,prurigo-nodularis,benign-epidermal,benign,prurigo-nodularis/9d5a90fa3f6934608add10e69800...
6,57885e3f5a3c043c3621a06bca196282.jpg,2,1,seborrheic-keratosis,benign-epidermal,benign,seborrheic-keratosis/57885e3f5a3c043c3621a06bc...
7,8adbbbc4e50a0df8b89710dfd495d3c5.jpg,5,4,eczema,inflammatory,non-neoplastic,eczema/8adbbbc4e50a0df8b89710dfd495d3c5.jpg
8,763ed484fcc50bf7b67cc44f95bac95e.jpg,-1,-1,folliculitis,inflammatory,non-neoplastic,folliculitis/763ed484fcc50bf7b67cc44f95bac95e.jpg
9,0198c74d604fde7055671d1b35869664.jpg,3,4,squamous-cell-carcinoma,malignant-epidermal,malignant,squamous-cell-carcinoma/0198c74d604fde7055671d...


In [7]:
def create_generator(dataframe, directory, batch_size=32, target_size=(128, 128)):
    """
   we didnt modify this function all that much 
    """
    # Fill in the correct flow_from_dataframe parameters
    generator = train_datagen.flow_from_dataframe(
        dataframe=dataframe,
        directory=directory,
        x_col='file_path',  # Use combined path
        y_col='encoded_label',
        target_size=target_size,
        batch_size=batch_size,
        class_mode='raw',
        validate_filenames=False  # Disable strict filename validation
    )
    return generator

# create train data 
label_encoder = LabelEncoder()
train_df['encoded_label'] = label_encoder.fit_transform(train_df['label'])

# split the data into training and validation sets
train_data, val_data = train_test_split(train_df, test_size=0.2, random_state=42)

# define image data generators for training and validation
train_datagen = ImageDataGenerator(rescale=1./255)
val_datagen = ImageDataGenerator(rescale=1./255)
# Create generators
train_generator = create_generator(train_data, train_dir)
val_generator = create_generator(val_data, train_dir)


Found 2288 non-validated image filenames.
Found 572 non-validated image filenames.


In [8]:
import os
from PIL import Image
import torch
from torch.utils.data import Dataset
from torch.utils.data import DataLoader
from torchvision import transforms

class CustomDataset(Dataset):
    def __init__(self, dataframe, base_directory, transform=None):
        self.dataframe = dataframe
        self.base_directory = base_directory  # Base directory path
        self.transform = transform

    def __len__(self):
        return len(self.dataframe)

    def __getitem__(self, idx):
        # Get the relative file path from the dataframe
        relative_path = self.dataframe.iloc[idx, 6]  # File path is in column 'file_path' (column index 6)
        img_path = os.path.join(self.base_directory, relative_path)  # Combine with base directory
        
        if not os.path.exists(img_path):
            raise FileNotFoundError(f"File not found: {img_path}")
        
        image = Image.open(img_path)  # Open the image
        
        if self.transform:
            image = self.transform(image)  # Apply transformations if provided

        # Get label (assuming label is in column 7 in dataframe)
        label = int(self.dataframe.iloc[idx, 7])  # Assuming the label is in column 7
        return image, label


#  create_loader function from btt template
def create_loader(dataframe, directory, batch_size=32, transform=None):
    dataset = CustomDataset(dataframe, directory, transform)
    loader = DataLoader(dataset, batch_size=batch_size, shuffle=True)
    return loader

# Tcreate the loaders like this
train_loader = create_loader(train_data, train_dir, batch_size=32, transform=None)
val_loader = create_loader(val_data, train_dir, batch_size=32, transform=None)


from torchvision import transforms

#  transformations for training and validation
train_transform = transforms.Compose([
    transforms.Resize((128, 128)),  # Resize images to 128x128
    transforms.RandomHorizontalFlip(),  # Randomly flip images horizontally
    transforms.ToTensor(),  # Convert PIL image to tensor
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])  
])

val_transform = transforms.Compose([
    transforms.Resize((128, 128)),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
])

# Base directory where images are stored
base_directory = '/kaggle/input/bttai-ajl-2025/train/train' 

# Create dataset and loaders for training and validation
train_dataset = CustomDataset(dataframe=train_df, base_directory=base_directory, transform=train_transform)
val_dataset = CustomDataset(dataframe=val_data, base_directory=base_directory, transform=val_transform)

train_loader = DataLoader(train_dataset, batch_size=32, shuffle=True)
val_loader = DataLoader(val_dataset, batch_size=32, shuffle=False)

## 4. Build the model


In [9]:
# build resnet18 model https://github.com/deepmancer/resnet-cifar-classification/blob/master/ResNet-18/CIFAR10_Classification.ipynb
class ResidualBlock(nn.Module):
    def __init__(self, in_len, out_len, stride=1):
        super(ResidualBlock, self).__init__()
        
        self.conv1 = nn.Conv2d(in_len, out_len, kernel_size=3, stride=stride, padding=1, bias=False)
        self.bn1 = nn.BatchNorm2d(out_len)
        self.conv2 = nn.Conv2d(out_len, out_len, kernel_size=3, stride=1, padding=1, bias=False)
        self.bn2 = nn.BatchNorm2d(out_len)
        
        layers = []
        if stride != 1 or in_len != out_len:
            layers.append(nn.Conv2d(in_len, out_len, kernel_size=1, stride=stride, bias=False))
            layers.append(nn.BatchNorm2d(out_len))
            self.shortcut = nn.Sequential(*layers)

        else:
            self.shortcut = nn.Sequential()
        

    def forward(self, x):
        out = self.conv1(x)
        out = self.bn1(out)
        out = F.relu(out)
        out = self.bn2(out)
        out = self.conv2(out)
        out += self.shortcut(x)
        out = F.relu(out)
        return out
    
    
class ResNet18(nn.Module):
    def __init__(self):
        super(ResNet18, self).__init__()

        self.conv1 = nn.Conv2d(3, 64, kernel_size=3, stride=1, padding=1, bias=False)
        self.bn1 = nn.BatchNorm2d(64)
        self.layer1 = nn.Sequential(
            ResidualBlock(64, 64, stride=1),
            ResidualBlock(64, 64, stride=1)
        )
        self.layer2 = nn.Sequential(
            ResidualBlock(64, 128, stride=2),
            ResidualBlock(128, 128, stride=1)
        )
        self.layer3 = nn.Sequential(
            ResidualBlock(128, 256, stride=2),
            ResidualBlock(256, 256, stride=1)
        )
        self.layer4 = nn.Sequential(
            ResidualBlock(256, 512, stride=2),
            ResidualBlock(512, 512, stride=1)
        )

        # Calculate the flattened size dynamically after the feature extraction part
        self.feature_size = self._get_feature_size()

        # modify last layer 
        self.linear = nn.Linear(self.feature_size, 21)

    def _get_feature_size(self):
        dummy_input = torch.zeros(1, 3, 128, 128) 
        out = self.conv1(dummy_input)
        out = self.bn1(out)
        out = F.relu(out)
        out = self.layer1(out)
        out = self.layer2(out)
        out = self.layer3(out)
        out = self.layer4(out)
        out = F.avg_pool2d(out, 4)
        out = torch.flatten(out, 1)
        return out.size(1)

    def get_feature_space(self, x):
        # Forward pass through the feature extractor
        out = self.conv1(x)
        out = self.bn1(out)
        out = F.relu(out)
        out = self.layer1(out)
        out = self.layer2(out)
        out = self.layer3(out)
        out = self.layer4(out)
        out = F.avg_pool2d(out, 4)  # Global average pooling
        out = torch.flatten(out, 1)  # Flatten the output to feed into the linear layer
        return out

    def forward(self, x):
        out = self.get_feature_space(x)  # Extract features using get_feature_space
        out = self.linear(out)  # Final fully connected layer
        return out

#instantiate
os.environ["CUDA_LAUNCH_BLOCKING"] = "1"
os.environ["TORCH_USE_CUDA_DSA"] = "1"
model = ResNet18()
model.to(device)

# optimize model 
optimizer = torch.optim.SGD(model.parameters(), lr=0.01, momentum=0.9, weight_decay=0.000125)
criterion = nn.CrossEntropyLoss()
lr_scheduler = torch.optim.lr_scheduler.CosineAnnealingLR(optimizer, T_max=200)

## 5. Train the Model


In [10]:
# TODO: Train your model here. 
#https://www.kaggle.com/code/ggsri123/implementing-resnet18-for-image-classification
def train_model(model, train_loader, val_loader, criterion, optimizer, lr_scheduler, num_epochs=50):
    best_accuracy = 0.0
    val_acc_history = []
    train_acc_history = []
    val_loss_history = []
    train_loss_history = []

    steps = ['train', 'validation']
    dataloaders = {'train': train_loader, 'validation': val_loader}

    print(f"Number of epochs = {num_epochs}")
    print('*********************')

    for epoch in range(num_epochs):
        print(f"Epoch {epoch + 1}/{num_epochs}")

        for step in steps:
            print(f"Step = {step}")
            pre_epoch(step, model)
            
            # Initialize accumulators
            running_loss = 0.0
            correct_number = 0
            total = 0

            # Iterate through batches
            for data, targets in dataloaders[step]:
                data, targets = data.to(device), targets.to(device)
                optimizer.zero_grad()

                # Forward pass
                outputs = model(data)
                batch_loss = criterion(outputs, targets)  # Rename loss to batch_loss
                running_loss += batch_loss.item() * data.size(0)  # Accumulate loss
                
                _, preds = torch.max(outputs, 1)
                correct_number += torch.sum(preds == targets.data)
                total += targets.size(0)

                # Backward pass for training step
                if step == 'train':
                    batch_loss.backward()
                    optimizer.step()

            epoch_loss = running_loss / total
            epoch_accuracy = correct_number.double() / total

            if step == 'validation':
                lr_scheduler.step()

                if epoch_accuracy > best_accuracy:
                    best_accuracy = epoch_accuracy
                    torch.save(model.state_dict(), 'best_model.pth')

                val_acc_history.append(epoch_accuracy)
                val_loss_history.append(epoch_loss)

            if step == 'train':
                train_acc_history.append(epoch_accuracy)
                train_loss_history.append(epoch_loss)

            print(f"{step.capitalize()} Loss: {epoch_loss:.4f} Accuracy: {epoch_accuracy:.4f}")

        print('*********************')

    return train_acc_history, val_acc_history, train_loss_history, val_loss_history


In [11]:
def pre_epoch(step, model):
    if step == 'train':
        model.train()
    elif step == 'validation':
        model.eval()
    else:
        raise 'Wrong Step'
        
torch.cuda.empty_cache()

num_epochs = 50

val_acc_history = []
train_acc_history = []
val_loss_history = []
train_loss_history = []

torch.cuda.empty_cache()
best_accuracy = 0.0
steps = ['train', 'validation']
dataloaders = {
    'validation': val_loader,
    'train': train_loader
}

print('number of epochs = {}'.format(num_epochs))
print('*********************')
for epoch in range(num_epochs):
    print('epoch = {}'.format(epoch))
    for step in steps: 
        print('step = {}'.format(step))
        pre_epoch(step, model)
        loss = 0.0
        correct_number = 0

        for data, targets in dataloaders[step]: 
            data = data.to(device)
            targets = targets.to(device)
            optimizer.zero_grad()
            with torch.set_grad_enabled(step == 'train'):
                outputs = model(data)
                loss = criterion(outputs, targets)
                _, preds = torch.max(outputs, 1)
                if step == 'train': 
                    loss.backward()
                    optimizer.step()
                    
            loss += loss.item() * data.size(0)
            correct_number += torch.sum(preds == targets.data)

        if step == 'validation':
            lr_scheduler.step()
            
        epoch_loss = loss / len(dataloaders[step].dataset)
        epoch_accuracy = correct_number.double() / len(dataloaders[step].dataset)


        if step == 'validation':
            if epoch_accuracy > best_accuracy:
                best_accuracy = epoch_accuracy
                torch.save(model.state_dict(), 'cifar10-resnet18.pth')
            val_acc_history.append(epoch_accuracy)
            val_loss_history.append(epoch_loss)               
            
        if step == 'train':
            train_acc_history.append(epoch_accuracy)
            train_loss_history.append(epoch_loss)
        
        print('loss = {:.4f} accuracy = {:.4f}'.format(epoch_loss, epoch_accuracy))

    print('*********************')

val_acc_history = [x.item() for x in val_acc_history]
train_acc_history = [x.item() for x in train_acc_history]
val_loss_history = [x.item() for x in val_loss_history]
train_loss_history = [x.item() for x in train_loss_history]

number of epochs = 50
*********************
epoch = 0
step = train
loss = 0.0156 accuracy = 0.1049
step = validation
loss = 0.3406 accuracy = 0.0909
*********************
epoch = 1
step = train
loss = 0.0135 accuracy = 0.1308
step = validation
loss = 0.1306 accuracy = 0.2098
*********************
epoch = 2
step = train
loss = 0.0116 accuracy = 0.1633
step = validation
loss = 0.1246 accuracy = 0.1591
*********************
epoch = 3
step = train
loss = 0.0116 accuracy = 0.1825
step = validation
loss = 0.1154 accuracy = 0.2255
*********************
epoch = 4
step = train
loss = 0.0119 accuracy = 0.1958
step = validation
loss = 0.1141 accuracy = 0.2360
*********************
epoch = 5
step = train
loss = 0.0094 accuracy = 0.2238
step = validation
loss = 0.1066 accuracy = 0.2622
*********************
epoch = 6
step = train
loss = 0.0126 accuracy = 0.2336
step = validation
loss = 0.0958 accuracy = 0.2815
*********************
epoch = 7
step = train
loss = 0.0115 accuracy = 0.2500
step = valid

In [29]:
torch.save(model.state_dict(), 'myResnet18.pth')
# SAVES MODEL 

# 6. Make predictions on test data


In [32]:

from torchvision import models

# Initialize the model (this should match the model you trained)
model = models.resnet18(pretrained=False)  
num_ftrs = model.fc.in_features
model.fc = nn.Linear(num_ftrs, 20)  

# Load the saved weights
model.load_state_dict(torch.load('myResnet18.pth'))

# Set the model to evaluation mode for inference
model.eval()

# Optionally, move the model to GPU if available
model.to(device)  # Ensure model is on the right device (CPU or GPU)


ResNet(
  (conv1): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
  (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (relu): ReLU(inplace=True)
  (maxpool): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
  (layer1): Sequential(
    (0): BasicBlock(
      (conv1): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU(inplace=True)
      (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    )
    (1): BasicBlock(
      (conv1): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU(inplace=True)
  

In [37]:
test_df.head()

,md5hash,fitzpatrick_scale,fitzpatrick_centaur,qc,ddi_scale
0,0844ae634f0e6e7ef1f73c2aeecbae0e.jpg,2,2,NaN,12
1,3b290d262098f761d719aa07cf36c040.jpg,4,3,NaN,34
2,cf561d08ac46d0fda678bff6621005ee.jpg,2,3,NaN,12
3,e6371069be05c6b0a95b4b3f1bacc9a5.jpg,4,3,NaN,34
4,f76cddb37265f97508f159078dcc7e7c.jpg,5,5,NaN,56


In [62]:
print(test_df['fitzpatrick_scale'].nunique())  # Check the number of unique classes in the test set
print(test_df.columns)
print(train_df['label'].unique())

7
Index(['md5hash', 'fitzpatrick_scale', 'fitzpatrick_centaur', 'qc',
       'ddi_scale', 'predictions'],
      dtype='object')
['prurigo-nodularis' 'basal-cell-carcinoma-morpheiform' 'keloid'
 'basal-cell-carcinoma' 'seborrheic-keratosis' 'eczema' 'folliculitis'
 'squamous-cell-carcinoma' 'actinic-keratosis' 'mycosis-fungoides'
 'acne-vulgaris' 'dyshidrotic-eczema' 'melanoma' 'epidermal-nevus'
 'malignant-melanoma' 'pyogenic-granuloma' 'dermatofibroma'
 'kaposi-sarcoma' 'acne' 'dermatomyositis'
 'superficial-spreading-melanoma-ssm']


In [85]:
from PIL import Image

# class names from label https://discuss.pytorch.org/t/understanding-training/135755/5  https://github.com/deepmancer/resnet-cifar-classification/blob/master/ResNet-18/CIFAR10_Classification.ipynb
class_names = [
    'prurigo-nodularis', 'basal-cell-carcinoma-morpheiform', 'keloid', 'basal-cell-carcinoma',
    'seborrheic-keratosis', 'eczema', 'folliculitis', 'squamous-cell-carcinoma', 'actinic-keratosis',
    'mycosis-fungoides', 'acne-vulgaris', 'dyshidrotic-eczema', 'melanoma', 'epidermal-nevus',
    'malignant-melanoma', 'pyogenic-granuloma', 'dermatofibroma', 'kaposi-sarcoma', 'acne',
    'dermatomyositis', 'superficial-spreading-melanoma-ssm'
]


# load saved model 
model = models.resnet18(pretrained=True)
num_ftrs = model.fc.in_features
model.fc = nn.Linear(num_ftrs, 20)
model.load_state_dict(torch.load('myResnet18.pth'))

# put model on device or else it won't work 
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model.to(device)
model.eval()  # Set model to evaluation mode

predictions = []

# make torch transform for test
test_transform = transforms.Compose([
    transforms.Resize((128, 128)),  # Resize images to 128x128
    transforms.ToTensor(),  # Convert PIL image to tensor
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])  
])

# test_df is above somewhere
test_dataset = CustomDataset(dataframe=test_df, base_directory='/kaggle/input/btt-aji/test/test', transform=test_transform)
test_loader = DataLoader(test_dataset, batch_size=32, shuffle=False)

# predictions loop https://pytorch.org/docs/stable/generated/torch.no_grad.html
with torch.no_grad(): 
    for inputs, _ in test_loader:
        inputs = inputs.to(device)
        outputs = model(inputs)  
        _, preds = torch.max(outputs, 1)  
        predictions.extend(preds.cpu().numpy())

# map thing
predicted_labels = [class_names[pred] for pred in predictions]

# delete '.jpg' from the MD5 hash only for the CSV export 
test_df['md5hash'] = test_df['md5hash'].apply(lambda x: x.replace('.jpg', ''))
test_df['predicted_label'] = predicted_labels

# Save the predictions to a CSV file
test_df[['md5hash', 'predicted_label']].to_csv('predictions.csv', index=False)

print("Predictions saved to predictions.csv")


Predictions saved to predictions.csv


In [86]:
test_df.head()

,md5hash,fitzpatrick_scale,fitzpatrick_centaur,qc,ddi_scale,predicted_label
0,0844ae634f0e6e7ef1f73c2aeecbae0e,2,2,NaN,12,mycosis-fungoides
1,3b290d262098f761d719aa07cf36c040,4,3,NaN,34,mycosis-fungoides
2,cf561d08ac46d0fda678bff6621005ee,2,3,NaN,12,squamous-cell-carcinoma
3,e6371069be05c6b0a95b4b3f1bacc9a5,4,3,NaN,34,mycosis-fungoides
4,f76cddb37265f97508f159078dcc7e7c,5,5,NaN,56,mycosis-fungoides


In [82]:
save_df = test_df[['md5hash', 'predicted_label']]

## 7. Generate Predictions

In [83]:
# download the test df into a csv

create_download_link(save_df)


In [90]:
# quick f1 score eval  
from sklearn.metrics import f1_score

predictions = []
labels1 = []

# make predictions on the val set same process as making predictions for the csv 
with torch.no_grad():  
    for inputs, labels in val_loader:
        inputs = inputs.to(device) 
        labels = labels.to(device)
        outputs = model(inputs)
        _, preds = torch.max(outputs, 1)
        # predictions 
        predictions.extend(preds.cpu().numpy()) 
        # labels 
        labels1.extend(labels.cpu().numpy()) 

# Calculate the F1 score
f1 = f1_score(all_labels, all_preds, average='weighted')  # You can use 'micro', 'macro', or 'weighted'
print(f'F1 Score on Validation Set: {f1}')

F1 Score on Validation Set: 0.0015076151406614112


In [ ]:
# the model with the highest f1 score appeared to be the resnet 101 with the most layers, so it is likely that the resnet variant will affect the accuracy of the model as more processing layers can allow for better accuracy. 